In [ ]:
#Does not work yet
raise Exception("Does not work yet")

In [ ]:
from datetime import datetime

import requests
import textwrap
def LLM(prompt, system_message, temperature, max_tokens, model,return_json=False):
    system_message = "Todays DateTime:"+datetime.now().strftime("%Y-%m-%d %H:00:00")+"\n"+system_message
    url = 'http://localhost:11435/api/passthrough/chat'
    headers = {'Content-Type': 'application/json'}
    payload = {
        'model': model,
        'cache': '1h',
        'keep_alive': '0m',
        'messages': [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': prompt}
        ],
        'temperature': temperature,
        'max_tokens': max_tokens,
        'return_json': return_json
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # Raise an HTTPError if the HTTP request returned an unsuccessful status code
        data = response.json()
        #print('Success:', data)
        return data
    except requests.exceptions.RequestException as error:
        print('Error:', error)
        raise




import json
def put_text_in_file(filename, text):
    """ Write text to a file in the jail directory """
    with open("./jail/" + filename, "w") as f:
        f.write(text)
    return "./jail/" + filename

import json
def create_directory(directory_name):
    """ Create a directory in the jail directory """
    import os
    os.makedirs("./jail/" + directory_name)
    return "./jail/" + directory_name

def indexJail():
    #return a list of all files in the jail directory recursively
    import os
    jail = "./jail/"
    files = []
    for root, directories, filenames in os.walk(jail):
        for filename in filenames:
            files.append(filename)
    return json.dumps(files)



my_tools = [put_text_in_file, create_directory]
tools=[]
tool_matchings = {}
# use openai function to json module
for tool in my_tools:
    tools.append({
        "name": tool.__name__,
        "description": tool.__doc__,
        "arguments": tool.__code__.co_varnames[:tool.__code__.co_argcount],
    })
    tool_matchings[tool.__name__] = tool
tools = json.dumps(tools, indent=2)
system_message=f"""

You work iterativly on tasks.
 
You have a filesystem at your disposal. its contents are:
{indexJail()}

You have access to the following tools:
{tools}
Use them to solve the current problem.

respond ONLY in object format of 
{{
    "tool": "name", 
    "arguments": ["arg1", "arg2",,,]
}}
"""
reply = LLM("""

write me a complete python tick tack toe game in the file main.py with a gui. it should have a good ai, you can define functions you need in other files.

""", system_message, 0.9, 60000, 'llama3.1',return_json=True)
print('\n'.join(textwrap.wrap(reply['message']['content'], width=50)))

    
reply_json = json.loads(reply['message']['content'])
print(reply_json)
tool = tool_matchings[reply_json['tool']]
arguments = reply_json['arguments']
print(tool(*arguments)) 